## Testing Models of Time Delay Function

Here, I want to test some simple analytical models, to see if my time delay function is making sense

In [6]:
import numpy as np

## Setting up the lens model

I take a simple point mass lens with line of sight effects

In [7]:
from lenstronomy.LensModel.lens_model import LensModel

# the lens model

lens_model_list = ['POINT_MASS', 'LOS']
lens_model_base = ['POINT_MASS']

kwargs_PM = {'theta_E': 2, 'center_x': 0, 'center_y': 0}
kwargs_LOS = {
    'kappa_od': 0.09, 'kappa_os': -0.1, 'kappa_ds': 0.08,
    'omega_od': 0.02, 'omega_os': -0.01, 'omega_ds': 0.01,
    'gamma1_od': 0.1, 'gamma2_od': -0.05,
    'gamma1_os': -0.1, 'gamma2_os': 0.03,
    'gamma1_ds': -0.02, 'gamma2_ds': 0.09}
kwargs_lens = [kwargs_PM, kwargs_LOS]
kwargs_base = [kwargs_PM]

lens_model_base = LensModel(lens_model_list=lens_model_base)
lens_model_class = LensModel(lens_model_list=lens_model_list)

No line-of-sight effects being added.
Adding LOS to the main lens.


## Time Delays

In what follows, I want to examine whether my time delay function produces the same results which are determined analytically 

In [8]:
#First check - ensuring that the kappas etc are inputted correctly

print("kappa_od = ", kwargs_LOS['kappa_od'])
print("omega_od = ", kwargs_LOS['omega_od'])
print("gamma1_od = ", kwargs_LOS['gamma1_od'])
print("gamma2_od = ", kwargs_LOS['gamma2_od'])
print('')
print("kappa_ds = ", kwargs_LOS['kappa_ds'])
print("omega_ds = ", kwargs_LOS['omega_ds'])
print("gamma1_ds = ", kwargs_LOS['gamma1_ds'])
print("gamma2_ds = ", kwargs_LOS['gamma2_ds'])
print('')
print("kappa_os = ", kwargs_LOS['kappa_os'])
print("omega_os = ", kwargs_LOS['omega_os'])
print("gamma1_os = ", kwargs_LOS['gamma1_os'])
print("gamma2_os = ", kwargs_LOS['gamma2_os'])


kappa_od =  0.09
omega_od =  0.02
gamma1_od =  0.1
gamma2_od =  -0.05

kappa_ds =  0.08
omega_ds =  0.01
gamma1_ds =  -0.02
gamma2_ds =  0.09

kappa_os =  -0.1
omega_os =  -0.01
gamma1_os =  -0.1
gamma2_os =  0.03


In [9]:
#checking that my matrices are returned correctly

A_od = np.array([[1-kwargs_LOS['kappa_od']-kwargs_LOS['gamma1_od'],-kwargs_LOS['gamma2_od']+kwargs_LOS['omega_od']],[-kwargs_LOS['gamma2_od']-kwargs_LOS['omega_od'],1-kwargs_LOS['kappa_od']+kwargs_LOS['gamma1_od']]])
A_os = np.array([[1-kwargs_LOS['kappa_os']-kwargs_LOS['gamma1_os'],-kwargs_LOS['gamma2_os']+kwargs_LOS['omega_os']],[-kwargs_LOS['gamma2_os']-kwargs_LOS['omega_os'],1-kwargs_LOS['kappa_os']+kwargs_LOS['gamma1_os']]])
A_ds = np.array([[1-kwargs_LOS['kappa_ds']-kwargs_LOS['gamma1_ds'],-kwargs_LOS['gamma2_ds']+kwargs_LOS['omega_ds']],[-kwargs_LOS['gamma2_ds']-kwargs_LOS['omega_ds'],1-kwargs_LOS['kappa_ds']+kwargs_LOS['gamma1_ds']]])

A_od_tsp = np.transpose(A_od)
A_ds_inv = np.linalg.inv(A_ds)
A_os_inv = np.linalg.inv(A_os)

print("A_od_tsp = ", A_od_tsp)
print("A_ds_inv = ", A_ds_inv)
print("A_os_inv = ", A_os_inv)
print('')

A_LOS = np.dot(np.dot(A_od_tsp,A_ds_inv),A_os)

print("A_LOS = ", A_LOS)

A_od_tsp =  [[0.81 0.03]
 [0.07 1.01]]
A_ds_inv =  [[1.07398568 0.09546539]
 [0.11933174 1.12171838]]
A_os_inv =  [[0.83388926 0.03335557]
 [0.01667779 1.00066711]]

A_LOS =  [[1.04599045 0.07603819]
 [0.21205251 1.13178998]]


This is perfectly consistent with my by-hand calculations

In [10]:
#next, checking that I arrive at the same distorted position
x_d, y_d = lens_model_class.lens_model.los.distort_vector(1,-1,
                                           kappa=kwargs_LOS['kappa_od'],
                                           omega=kwargs_LOS['omega_od'],
                                           gamma1=kwargs_LOS['gamma1_od'],
                                           gamma2=kwargs_LOS['gamma2_od'])

print('position at which the light leaves the lens plane = ', x_d, y_d)

position at which the light leaves the lens plane =  0.74 -0.98


Once again matches my calculations

In [11]:
#checking that the lensing potential is calculated correctly

psi_eff = lens_model_base.potential(x_d, y_d,kwargs_base)              #evaluating the potential at the distorted position

print("effective potential = ", psi_eff)

effective potential =  0.8215685391715289


Matches my calculations

In [12]:
#checking that the los deflection angle and source position are calculated correctly

alpha_x, alpha_y = lens_model_class.alpha(1,-1,kwargs_lens)
x_source, y_source = lens_model_class.ray_shooting(1,-1,kwargs_lens)

print("alpha = ", alpha_x, alpha_y)
print("beta = ", x_source, y_source)

alpha =  1.8130503978779835 -2.515809018567638
beta =  -0.8130503978779835 1.515809018567638


Matches my calculations

In [13]:
#checking that the source position modified by A_os_inv is calculated correctly

b_x = A_os_inv[0][0]*x_source + A_os_inv[0][1]*y_source
b_y = A_os_inv[1][0]*x_source + A_os_inv[1][1]*y_source

print("A_os_inv*beta = ", b_x, b_y)

A_os_inv*beta =  -0.6274333198259489 1.503260352171119


Matches my calculations

In [14]:
#checking that the displacement angle is calculated correctly

f_x = 1 - b_x
f_y = -1 - b_y 

print("alpha' = ", f_x, f_y)

alpha' =  1.6274333198259487 -2.503260352171119


Matches my calculations

In [15]:
#checking that the displacement angle modified by Gamma_LOS is calculated correctly
a_x = A_LOS[0][0]*f_x + A_LOS[0][1]*f_y
a_y = A_LOS[1][0]*f_x + A_LOS[1][1]*f_y

print("Gamma_LOS*alpha = ", a_x, a_y)

Gamma_LOS*alpha =  1.511936339522546 -2.4880636604774518


Matches my calculations

In [16]:
#testing the Fermat Potential function itself

geometry = (f_x*a_x + f_y*a_y) / 2

fermat_1 = geometry - psi_eff
print("Fermat Potential 1 = ", fermat_1)

fermat_pot_test2 = lens_model_class.fermat_potential(1,-1,kwargs_lens)
print("Fermat Potential 2 = ", fermat_pot_test2)

Fermat Potential 1 =  3.5228548065012806
Fermat Potential 2 =  3.5228548065012806


Matches my calculations